In [1]:
import os
from tqdm import tqdm
from qecsim.models.rotatedplanar import RotatedPlanarCode
from src.qiskit_noise_models import get_noise_model
from src.circuit_builder import build_qiskit_circuit
from qiskit_aer import AerSimulator
from qiskit.result import Result
import jax.numpy as jnp
from jax import random
import json

key = random.key(0) # Only affects the random deformations and basis not the circuit noise

# Collect data

In [2]:
data_points = 1000
rounds = 20
code_width, code_height = 5, 5
noise_model_name = "spin"

code = RotatedPlanarCode(code_width, code_height)
backend = AerSimulator(method='stabilizer', noise_model=get_noise_model(noise_model_name), shots=1)

experiments = []
for _ in tqdm(range(data_points), "Generating data", ncols=200):
    key, deformation_key, basis_key = random.split(key, 3)
    allowed = jnp.array([1,2])
    # Randomly choose a deformation for each data qubit
    deformation = random.choice(deformation_key, allowed, shape=(code.n_k_d[0],), replace=True)
    basis = ['+X', '-X', '+Z', '-Z'][random.randint(basis_key, (), 0, 4)]
    circ = build_qiskit_circuit(code, deformation, rounds, basis=basis)
    results: Result = backend.run(circ, memory=True).result()
    # Process results as needed
    syndromes_txt = results.get_memory()[0]
    syndromes = jnp.array([[int(bit) for bit in bits] for bits in syndromes_txt.split(' ')])
    data = results.data()
    observable = [float(data[f"observable_R{i}"][0]) for i in range(rounds)]
    experiments.append({
        "deformation": deformation.tolist(),
        "basis": basis,
        "observable": observable,
        "syndromes": syndromes.tolist(),
    })

Generating data: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [07:52<00:00,  2.12it/s]


# Save the data

In [3]:
# Helper function to format numbers with scientific prefixes
def scientific_prefix_formatter(num: int) -> str:
    """Formats a number with scientific prefixes for better readability.

    Args:
        num (int): The number to format.

    Returns:
        str: The formatted string with scientific prefix.
    """
    prefixes = {1e9: 'G', 1e6: 'M', 1e3: 'k', 1: '', 1e-3: 'm', 1e-6: 'µ', 1e-9: 'n'}
    for factor, prefix in prefixes.items():
        if num >= factor:
            formatted_num = num / factor
            if formatted_num.is_integer():
                return f"{int(formatted_num)}{prefix}"
    return str(num)

scientific_prefix_formatter(data_points)

'1k'

In [4]:
save_as = f"qiskit_{noise_model_name}_{code_width}x{code_height}_{scientific_prefix_formatter(data_points)}_r{rounds}"
if os.path.exists(f"../data_sets/{save_as}.json"):
    raise FileExistsError(f"File ../data_sets/{save_as}.json already exists. Please choose a different name to avoid overwriting existing data or manually delete the file.")
with open(f"../data_sets/{save_as}.json", "w") as f:
    json.dump(experiments, f, indent=4)
print(f"Data saved to data_sets/{save_as}.json")

Data saved to data_sets/qiskit_spin_5x5_1k_r20.json
